# Case Study: Contrail Segmentation 

This notebook presents a set of exercises in the form of a case study on segmenting contrails from satellite image (also referred to as remote sensing data). In it you will be introduced to a data science scenario you will have to tackle, challenge the issue of data scarcity, talk about the ethics of the usage of your model, and how to verify your model works on a technical level.

*Prerequisties*:
- Basic machine learning knowledge 

*The goal by the end of this notebook is for students to:*
- Gain an exposure to a new application of artificial intelligence - extracing semantics from satellite images.
- Understand when they can consider the specifics of their data and task.
- Think about the implications of their machine learning beyound the technical - what will be my machine learning be used for? 
- Understand how to evaluate machine learning tasks based on the goal they are solving - how more complicated tasks need more complicated metrics.
- Become more informed as engineers in their place in the whole machine learning pipeline.

*The notebook is loosely based on the paper "Few-Shot Contrail Segmentation in Remote Sensing Imagery With Loss Function in Hough Space" by Junzi et al. It focuses on creating an automatic segmentation procedure for contrails when using few data samples by taking advantage of what we know about the problem.*



##  What is a contrail?

Contrails are line-shaped clouds produced by aircraft engine exhaust due to changes in air pressure, typically at aircraft cruising altitudes several kilometres/miles above the Earth's surface. (Definition - https://en.wikipedia.org/wiki/Contrail)

<p align="center">
  <img src="https://raw.githubusercontent.com/vdakov/contrail-segmentation/main/images/what-is-a-contrail.png" width="800"/>
</p>



[Image Source](https://www.google.com/url?sa=i&url=https%3A%2F%2Fsites.research.google%2Fgr%2Fcontrails%2F&psig=AOvVaw128OqAWqIAmXihrXxVq9OS&ust=1763826829380000&source=images&cd=vfe&opi=89978449&ved=0CBEQjRxqFwoTCJjx8JHNg5EDFQAAAAAdAAAAABAE)

## Why does it matter?

The Sun emits solar radiation towards the Earth that the ground traditionally reflects back. However, the formation of the ice crystals in contrails creates a dense enough "shield" to reflect a part of them back. This impacts the amount of radiation trapped around the Earth and thus contributes to the temperature. They also have a potential cooling effect on sun rays getting reflected back towards the Sun, though this effect is currently estimated to be smaller.


## Problem Definition

The contrails reflect light and cool during the day, but trap heat during the night. This becomes an issue when contrails stick around for hours in specific regions – ISSRs – 80% of contrail warming  

## The Challenge

Contrails are already being measured by aircraft sensors and estimation models – as the ones responsible if something goes wrong. In the future, however, with issues like global warming, governments may want to do contrail reporting themselves. While there are already ways this being  done via sensors and other models, they are provided by the airlines. The government needs an independent version of these reports.

## Can we solve this with AI?

Yes, using satellite images and neural networks, we can make a model to segment the contrails from arbitrary images. Good news: using satellite 
images and neural networks, we can make a model to segment the contrails from arbitrary images 

###  What is Contrail Segmentation?

Contrail segmentation is a type of semantic segmentation, where the goal is to identify which parts of a satellite image contain contrails.

### Segmentation 

In normal image classification, you assign a single label to an entire image.
In segmentation, you label every pixel so the model can understand what objects appear and where they are.

### Contrail Segmentation

For contrail segmentation, there are only two pixel types:

- pixels that are part of a contrail
- pixels that are background

The model takes a satellite image and produces another image (a mask) showing where contrails are located.


<div style="display: flex; justify-content: space-around;">
   
   <img src="https://raw.githubusercontent.com/vdakov/contrail-segmentation/main/contrail-seg/data/goes/florida/image/florida_2020_03_05_0101.png" width="400"/>
   <img src="https://raw.githubusercontent.com/vdakov/contrail-segmentation/main/contrail-seg/data/goes/florida/mask/florida_2020_03_05_0101.png" width="400">

</div>

[Image Source](https://www.kaggle.com/competitions/google-research-identify-contrails-reduce-global-warming/data)


Below are listed some example images of contrail images taken from different satellites and bands: 


<div style="display: flex; justify-content: space-around;">
  <img src="https://raw.githubusercontent.com/vdakov/contrail-segmentation/main/images/1-MeteoSat 11.png" height="350" width="350"/>
  <img src="https://raw.githubusercontent.com/vdakov/contrail-segmentation/main/images/2-NASA Terra.jpg" height="350" width="350"/>
  <img src="https://raw.githubusercontent.com/vdakov/contrail-segmentation/main/images/3-NOAA Suomi-NPP.jpg" height="350" width="350"/>
</div>



## The Scenario 
You are a data scientist at VeryCoolCompany Ltd. And the government has asked you to try to make this for them.  In this notebook you will tackle four challenges in regards to this: 

- **Data issues**: data is expensive, hard to get, label, etc. It costs money for your company. Sometimes even your datasets are samey. Different types of contrails 
- **Scientific Uncertainty**: Uncertainty in CO2 forcing, up to 70% of the actual impact on contrails
- **Model Quality and Uncertainty**
- **Policy Issues**: How should the government employ this tool you provide them?  Even if a policy is decided, how can it be enforced – globally, locally?  

## Project Setup

### IF LOCAL: Create an Anaconda Environment and Download Requirements
If you haven't already setup the codebase following the instructions of the README, you can do so here. Otherwise, you can skip running this.

In [ ]:
# %conda create -y -n contrail-project python=3.12
# %conda activate contrail-project
# %conda install -y pip
# %pip install -r requirements.txt

### IF ON COLAB: Download Repository 




In [ ]:
import os

def is_colab():
    return "COLAB_GPU" in os.environ or "google.colab" in str(get_ipython())

def is_kaggle():
    return os.path.exists("/kaggle")

repo_url = "https://github.com/vdakov/contrail-segmentation.git"

if is_colab():
    # Colab-specific setup

    
    !git clone --branch dev --single-branch {repo_url}
    !ls contrails
    
    os.chdir("contrails")
    !pip install --upgrade pip
    !pip install lightning segmentation_models_pytorch
    !pip install -r requirements.txt
    
    os.chdir("data_util/AudioMNIST")
    !git clone https://github.com/Jakobovski/free-spoken-digit-dataset.git
    os.chdir("../../")

elif is_kaggle():
    # Kaggle-specific setup
    print("Running on Kaggle. Installing requirements...")
    
    os.chdir("/kaggle/working")  # Ensure working directory
    
    !git clone --branch dev --single-branch {repo_url}
    !ls contrails
    
    os.chdir("contrails")
    !pip install lightning segmentation_models_pytorch ipyml
    
    
    os.chdir("data_util/AudioMNIST")
    !git clone https://github.com/Jakobovski/free-spoken-digit-dataset.git
    os.chdir("../../")

else:
    print("Not running on Colab or Kaggle. Skipping installation steps.")


### Get Imports

In [ ]:
import os
import random
import numpy as np
import torch
import os
from data_util.AudioMNIST.load_data import load_audio_data, load_audio_data_noisy
from visualization.show_audio import visualize_waveform_and_play_audio
from data_util.AudioMNIST.split_data import get_balanced_indices_by_targets
from visualization.show_metrics import plot_accuracy_vs_subset_size
from torch import nn, optim
from tqdm import tqdm
from data_util.MNIST.raw.mnist_loading import load_mnist_data
from models.simpleCNN import SimpleCNN
from visualization.show_images import visualize_pytorch_dataset_images
import albumentations as A
from visualization.interactive_menu import retrieve_mnist_menu, AlbumentationsTransform
import sys
import cv2
sys.path.append("contrail-seg")
from train import train_contrail_network
from evaluate import evaluate_contrail_model, plot_pr_curve
seed=42
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

## Which Dataset to Choose? 

You are a data scientist. All of your issues start at the dataset you will use for the task at hand. Wherever you go, you first need to consider your domain. In this case, the domain is satellite images and flight trajectories. You need to consider which satellite to collect images from. 

<!-- <iframe src="https://drive.google.com/file/d/1J7-wbA8b5QWF_o-rdsUfArvNTpirGLac/preview" width="1000" height="600" allow="autoplay"></iframe> -->

<img src="https://raw.githubusercontent.com/vdakov/contrail-segmentation/dev/images/landsat-vs-goes.png" width= "1000">


<!-- https://drive.google.com/file/d/1J7-wbA8b5QWF_o-rdsUfArvNTpirGLac/view?usp=drive_link) -->

[Image Source 1](https://www.kaggle.com/competitions/google-research-identify-contrails-reduce-global-warming/data)
[Image Source 2](https://console.cloud.google.com/storage/browser/landsat_contrails_dataset)


We present you with two options: LandSAT and GEOS. In the following section, you have to answer the following overarching question, and then a series of questions: 

#### Does it matter what satellite we pick? 

Base your answer on this list of characteristics, as well as the resources provided below:
**LandSat Characteristics**
- Low Earth Orbit 
- Orbits once every 16 days 
- 8 channels 
- Lower resolution
For more, please consult https://landsat.gsfc.nasa.gov/satellites/landsat-8/. 

**GEOS Characteristics**  
- Higher Orbit 
- Higher Resolution 
- Continuous monitoring over an area

For more, please consult https://science.nasa.gov/mission/goes/. 


<div style=" font-family: Arial, sans-serif; background: #00e5ff; color: #0b1220; padding: 15px 20px; border-radius: 12px; line-height: 1.6; max-width: 1000px; margin-bottom: 20px; "> 

**Question 1:** What are the tradeoffs between the two datasets?

- To understand this, look up the datasets online: (Landsat: https://landsat.gsfc.nasa.gov/satellites/landsat-8/; GOES: https://science.nasa.gov/mission/goes/.)
)
- How did the data collection differ? 
- What variations (technical differences) do the different datasets have?
- Could there be any bias in either dataset? Why?
- In the context of the VeryCoolCompany; do you see any issues with either dataset?  
--- 

**Answer:**

<!-- START OF ANSWER -->
<!-- END OF ANSWER --> 

</div>

# Data Scarcity - Data Augmentation

Scarcity is an issue everybody who works with data faces eventually. Data is expensive to acquire, hard to label, and a good machine learning model benefits from as much .(good) data as possible. 

Let's take the Google GOES dataset for example. Details for it are provided in https://www.kaggle.com/competitions/google-research-identify-contrails-reduce-global-warming/data. How much do you think it costs?





<div style=" font-family: Arial, sans-serif; background: #00e5ff; color: #0b1220; padding: 15px 20px; border-radius: 12px; line-height: 1.6; max-width: 1000px; margin-bottom: 20px; "> 


**Question 2:** Open the dataset link above. Examine the dataset and try to estimate its costs. Think about the number of images, how they get labeled, and how much that might cost. It's fine if you make assumptions - this is not a financial analysis.

--- 

**Answer:**

<!-- START OF ANSWER -->
<!-- END OF ANSWER -->  

</div>

Your boss at VeryCoolCompany saw this cost and got horrified at it! This is more than the project would cost otherwise. As such, we will use a very small dataset, hand-labeled dataset of Florida and San Francisco (about 40 images total). You have to find a way to make the contrail detection task work with very little data.


**Data Augmentation** is a way to reduce overfitting on small datasets, and tackle all of this issue. 

It is a set of transformations on the data samples that do not change its labels. The transformations change depending on the data being modified - every task has different semantics. The idea is to make the models focus on the essense of the data, rather than arbitrary aspects of it (noise). These transformations may very during training.

**This section introduces data augmentation in two modalities: audio and images. Later, we will also show the value of that on our task of contrails.**

To illustrate the point of data augmentation, we will guide you through two examples: on an audio and on an image data set. One is a fully-guided example, the other ones need to have either code filled in and/or theoretical questions answered. Finally, this is all related to the contrail case. 


### Audio

Here we show a simple task: audio classification. Given an audio file, can we understand automatically what number is being said based on the recording? For this purpose, we are going to use a subset of the FSDD dataset, and evaluate performance on it both with and without data augmentation. The dataset contains recordings of spoken numbers, similar to MNIS but for audio.

Free Spoken Digit Dataset (FSDD)- https://github.com/Jakobovski/free-spoken-digit-dataset


In [ ]:
max_samples = 100
X_audio, y_audio, wavs = load_audio_data('data_util/AudioMNIST/free-spoken-digit-dataset/recordings', max_samples=max_samples)
X_audio_train, X_audio_test, y_audio_train, y_audio_test = train_test_split(X_audio, y_audio, test_size=0.2, random_state=42)

#### Original Audio Samples

An audio file is created when an analog processor converts sound waves into a sequence of amplitudes and spectrograms. They respectively give an idea of when the volume and frequency are higher. When it is played, it is also just an array of amplitudes, showing the intensity. By looking at the waveform graphs, you can clearly discern when someone is speaking, and also play the audio.

In [ ]:
visualize_waveform_and_play_audio(y_audio[:3], wavs[:3])

#### Augmented Audio Samples

This signal seems to have lots of amplitudes, right, and varying length, based on the person, voice, etc. We have prepared an MNIST dataset, of numbers. It turns out that there are transformations we can do that will not change the label - in other words, the semantics are preserved. For example:
- speeding up or slowing down the speech still gives the same number 
- shifting the pitch as well
- adding noise, making a worse recording should not change the outcome as well

This is all a form of **data augmentation** - changes to our data that does not impact how our model should treat it, while being more adversarial and challenging for it. Below we have prepared one of the examples previously listed - adding Gaussian noise. 

Below is a cell performing all of these perturbations as a function. We have already pre-loaded it for you on the same dataset.

Take a listen and see if the numbers are still discernable. Also, try to examine how the signal looks visually different. Is there anything different about the wav file and the spectrograms? Take a look and answer the questions below.

In [ ]:
def train_test_split(X, y, test_size=0.2, random_state=None, shuffle=True):
    if random_state is not None:
        np.random.seed(random_state)
    
    X = np.array(X)
    y = np.array(y)
    n_samples = X.shape[0]
    
    indices = np.arange(n_samples)
    if shuffle:
        np.random.shuffle(indices)
    
    test_count = int(n_samples * test_size)
    
    test_indices = indices[:test_count]
    train_indices = indices[test_count:]
    
    X_train, X_test = X[train_indices], X[test_indices]
    y_train, y_test = y[train_indices], y[test_indices]
    
    return X_train, X_test, y_train, y_test


def augmentation_function(x, pitch_scale_min=0.9, pitch_scale_max=1.1, slow_down_factor=0.85, sigma=0.001):
    import numpy as np
    x = x * np.random.uniform(pitch_scale_min, pitch_scale_max) 
    x = np.interp(np.arange(0, len(x), slow_down_factor), np.arange(0, len(x)), x)
    x = x + np.random.normal(0, sigma, len(x))  
    return x

X_audio_augmented, y_audio_augmented, wavs_augmented = load_audio_data_noisy('data_util/AudioMNIST/free-spoken-digit-dataset/recordings', augmentation_function, max_samples=max_samples)
X_audio_train_augmented, _, y_audio_train_augmented, _ = train_test_split(X_audio_augmented, y_audio_augmented, test_size=0.2, random_state=42)

In [ ]:
visualize_waveform_and_play_audio(y_audio_augmented[1:4:2], wavs_augmented[1:4:2])

<div style="
  font-family: Arial, sans-serif;
  background: #00e5ff;
  color: #0b1220;
  padding: 15px 20px;
  border-radius: 12px;
  line-height: 1.6;
  max-width: 1000px;
">

**Question 3:**  
How is the amplitude (the left plot) different for the augmented samples – why?  

---

**Answer:**  
<!-- START OF ANSWER -->
<!-- END OF ANSWER -->
</div>


<div style="
  font-family: Arial, sans-serif;
  background: #00e5ff;
  color: #0b1220;
  padding: 15px 20px;
  border-radius: 12px;
  line-height: 1.6;
  max-width: 1000px;
">

**Question 4:**  
What would happen if we increase $\sigma$ to 100? What about `slow_down_factor` to 0.001? What if we set min and max pitch the same? Is there anything in common with these proposed augmentations?

---

**Answer:**  
<!-- START OF ANSWER -->
<!-- END OF ANSWER -->
</div>


<div style="
  font-family: Arial, sans-serif;
  background: #00e5ff;
  color: #0b1220;
  padding: 15px 20px;
  border-radius: 12px;
  line-height: 1.6;
  max-width: 1000px;
">

**Question 5:**  
If you were to make a new data augmentation for audio, what would it be? 

---

**Answer:**  
<!-- START OF ANSWER -->
<!-- END OF ANSWER -->
</div>




## Images Data Augmentation - MNIST

In this part of the notebook, we will show you a more popular example of data augmentation - namely on images and the MNIST Dataset. It is where data augmentation is more commonly applied in industry and more easily understood. 


Please run all code cells below, and you will end up with an interactive menu on the MNIST dataset. There you can play around with different data augmentation and have them visualized in real time. Then, run through the theoretical exercises and try to continue by improving the list of augmentations provided.


In [ ]:
def train(model, train_dataloader, val_dataloader, epochs=3):
    model.train()
    optimizer = optim.Adam(model.parameters())
    criterion = nn.CrossEntropyLoss()
    train_losses, val_losses = [], []
    for epoch in tqdm(range(epochs)):
        train_loss = []
        for images, labels in train_dataloader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss.append(loss.item())
        train_losses.append(np.mean(train_loss))
        val_loss = 0
        model.eval()
        with torch.no_grad():
            for images, labels in val_dataloader:
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
        val_losses.append(val_loss / len(val_dataloader))
    return train_losses, val_losses

def test(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in dataloader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

In [ ]:
menu = retrieve_mnist_menu()
menu

## Exercise: Creating Your Own Data Augmentation Pipeline

The time has come. After seeing our examples and playing around with how the given numbers look around, we want you to arrange your own set of augmentations. Your task is to find an augmentation to boost the performance on a training and validation set on a subset of data we offer you. 

You will do all of this in the `transform_aug` variables, which takes in a list of augmentations. This list is currently empty (except for the normalization step, which we leave as an exercise for the reader to know why it is there). Throughout this notebook we have been using the *Albumentations* library to do our data augmentations. This will continue to be the case here. You are free to select from any of the following functions there, and adjust their parameters. Namely:
- `A.Rotate(float rotate, float p)`  
- `A.Affine(float translate_x, float translate_y, float scale, float p)`  
- `A.GaussianBlur(int gaussian_kernel_size, float blur_sigma, float p, bool gaussian_blur)`  
- `A.HorizontalFlip(float p, bool horizontal_flip)`  
- `A.VerticalFlip(float p, bool vertical_flip)`  
- `A.RandomBrightnessContrast(float p, bool random_brightness_contrast)`  
- `A.RGBShift(float p, bool rgb_shift)`  

**Important**: Please take note that sometimes the function can take in both a number, a tuple, and a list!! 

The API of the augmentations is provided on this link https://albumentations.ai/docs/api-reference/. Be careful - the optimal combination may not be all of those augmentations at once, or with all of the parameters! 


In [ ]:
## START ANSWER 
p=0.25
## END ANSWER

transform_aug = AlbumentationsTransform(A.Compose([
            # Add your chosen augmentations here, e.g.:
            # A.Rotate(limit=(float rotate), float p), p=0.
            ## START OF ANSWER 
            ## END OF ANSWER 
            A.Normalize(mean=0.5, std=0.5, max_pixel_value=255.0),
            
], seed=42))

<div style="
  font-family: Arial, sans-serif;
  background: #00e5ff;
  color: #0b1220;
  padding: 15px 20px;
  border-radius: 12px;
  line-height: 1.6;
  max-width: 1000px;
">

**Question 6:**  
How often would you choose to apply the data augmentation with the parameter **p**? Justify. Think what happens if the values end up between 0 and 1.

---

**Answer:** 
<!-- START OF ANSWER --> 
<!-- END OF ANSWER -->

</div>


In [ ]:
transform_no_aug = AlbumentationsTransform(A.Compose([
    A.Normalize(mean=0.5, std=0.5, max_pixel_value=255.0),
], seed=42))

## Dataset Parameters and Visualization

Here we set you the subset size you will be working on with your data augmentation from the code cell above. The expectation is that accuracy will increase for the correct set of data augmentation. Play around it - adjust the size of the dataset if you wish to or the samples per class, which attemt to balance out your distribution.

In [ ]:
num_classes = 10
epochs = 100 #WARNING - IF TOO LOW, MAY NOT WORK

trainset_no_aug, valset_no_aug, _, trainloader_no_aug, valloader_no_aug, testloader = load_mnist_data(train=True,data_augmentations=transform_no_aug,  subset_size=400, )
trainset_aug, valset_aug, _,  trainloader_aug, valloader_aug, _ = load_mnist_data(train=True, data_augmentations=transform_aug, subset_size=400)

visualize_pytorch_dataset_images(trainset_no_aug, "No Data Augmentation")
visualize_pytorch_dataset_images(trainset_aug, "With Data Augmentation")

In [ ]:
model_no_aug = SimpleCNN()
losses_no_aug_train, losses_no_aug_val = train(model_no_aug, trainloader_no_aug, valloader_no_aug, epochs=epochs)
acc_no_aug = test(model_no_aug, testloader)

model_aug = SimpleCNN()
losses_aug_train, losses_aug_val = train(model_aug, trainloader_aug, valloader_no_aug, epochs=epochs)
acc_aug = test(model_aug, testloader)

<div style="
  font-family: Arial, sans-serif;
  background: #00e5ff;
  color: #0b1220;
  padding: 15px 20px;
  border-radius: 12px;
  line-height: 1.6;
  max-width: 1000px;
">

## Challenge: Accuracy > 88%

We have provided you a data augmentation setup for a limited subset size. You have the full capability to improve performance on the MNIST task. We challenge you to reach an accuracy of above 88%. You can add whatever data augmentations you want, change the probabilities of the augmentation **p**, or increase training time. To get test sense performance, run the cell below.



</div>


<div style="
  font-family: Arial, sans-serif;
  background: #00e5ff;
  color: #0b1220;
  padding: 15px 20px;
  border-radius: 12px;
  line-height: 1.6;
  max-width: 1000px;
">

**Question 7:**  
What data augmentations did you choose and why? Are there any data augmentations you did NOT choose? If so, why? Feel free to come back to this questions after experiemnting.

---

**Answer:**  
<!-- START OF ANSWER -->
<!-- END OF ANSWER -->

</div>


In [ ]:
from visualization.show_metrics import compare_two_losses
import matplotlib.pyplot as plt

print(f"Accuracy without augmentation: {acc_no_aug:.4f}")
print(f"Accuracy with augmentation: {acc_aug:.4f}")

def compare_two_losses(loss_a, loss_a_val, loss_a_label, loss_b, loss_b_val, loss_b_label):


    plt.figure(figsize=(12, 6))
    plt.plot(loss_a, label=loss_a_label)
    plt.plot(loss_a_val, label=loss_a_label + ' (val)')
    plt.plot(loss_b, label=loss_b_label)
    plt.plot(loss_b_val, label=loss_b_label + ' (val)')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    plt.title('Loss Comparison')
    plt.show()
    

def plot_accuracy_comparison(acc_no_aug_list, acc_aug_list):
    accuracies = [acc_no_aug_list[-1], acc_aug_list[-1]]
    labels = ['No Data Augmentation', 'With Data Augmentation']

    plt.figure(figsize=(12, 6))
    plt.bar(labels, accuracies, color=['red', 'green'])
    plt.ylim([0, 1])
    plt.ylabel('Accuracy')
    plt.title('Model Accuracy Comparison')

    for i, acc in enumerate(accuracies):
        plt.text(i, acc + 0.02, f"{acc:.4f}", ha='center')

    plt.show()

    
compare_two_losses(losses_no_aug_train, losses_no_aug_val, 'No Data Augmentation', losses_aug_train, losses_aug_val, "With Data Augmentation")
plot_accuracy_comparison([acc_no_aug], [acc_aug])

<div style="
  font-family: Arial, sans-serif;
  background: #00e5ff;
  color: #0b1220;
  padding: 15px 20px;
  border-radius: 12px;
  line-height: 1.6;
  max-width: 1000px;
">

**Question 8:**  
Examine the training loss over different epochs (both validation and training) on the augmented and non-augmented data. What difference do you see? Why do you see it? What about the validation loss?

---

**Answer:**  
The augmented training loss is much less stable than the non-augmented one. This makes sense as augmentations change over forward passes and the task is overall much harder. The validation loss on the non-augmented set starts rising. This indicates overfitting.
</div>




In [ ]:
subset_sizes = [80, 160, 320, 640, 1280]  # Different subset sizes to try
epochs = 100 #WARNING - IF TOO LOW, MAY NOT WORK

acc_no_aug_list = []
acc_aug_list = []

for subset_size in subset_sizes:
    trainset_no_aug, valset_no_aug, _, trainloader_no_aug, valloader_no_aug, testloader = load_mnist_data(train=True,data_augmentations=transform_no_aug,  subset_size=subset_size, )
    trainset_aug, valset_aug, _,  trainloader_aug, valloader_aug, _ = load_mnist_data(train=True, data_augmentations=transform_aug, subset_size=subset_size)
    
    model_no_aug_curr = SimpleCNN()
    losses_no_aug_train, losses_no_aug_val = train(model_no_aug_curr, trainloader_no_aug, valloader_no_aug, epochs=epochs)
    acc_no_aug_curr = test(model_no_aug_curr, testloader)

    model_aug_curr = SimpleCNN()
    losses_aug_train, losses_aug_val = train(model_aug_curr, trainloader_aug, valloader_no_aug, epochs=epochs)
    acc_aug_curr = test(model_aug_curr, testloader)

    acc_no_aug_list.append(acc_no_aug_curr)
    acc_aug_list.append(acc_aug_curr)

plot_accuracy_vs_subset_size(
    subset_sizes=subset_sizes,
    acc_no_aug=acc_no_aug_list,
    acc_with_aug=acc_aug_list,
    title='Performance: Impact of Data Augmentation vs. Subset Size'
)

<div style="
  font-family: Arial, sans-serif;
  background: #00e5ff;
  color: #0b1220;
  padding: 15px 20px;
  border-radius: 12px;
  line-height: 1.6;
  max-width: 650px;
">

**Question 9:**  
Reason about the performance on the augmented data over the different subset sizes on the plot.

---

**Answer:**  
<!-- START OF ANSWER -->
<!-- END OF ANSWER -->
</div>


### Contrail Application

Now comes the time to apply our data augmentations to real data, not just a toy problem like MNIST. 

We are in luck, however! We can (mostly) just reapply all of the data augmentations we have done to MNIST. The following section details how data augmentation is applied and lets you play with another set of augmentations to improve contrail performance. 

Alternatively, you could also take a look at our set of data augmentatiosn they are not necessarily the best set of augmentations for all cases. Please answer all of the theoretical questions, and run through the code. Our configuration of data augmentations is not necessarily optimal.

In [ ]:
from visualization.interactive_menu import retrieve_contrails_menu
menu = retrieve_contrails_menu()
menu

<div style="
  font-family: Arial, sans-serif;
  background: #00e5ff;
  color: #0b1220;
  padding: 15px 20px;
  border-radius: 12px;
  line-height: 1.6;
  max-width: 650px;
">

**Question 10:**  
Is the data augmentations on the contrails different algorithmically? What has changed?

---

**Answer:**  
<!-- START OF ANSWER -->
<!-- END OF ANSWER -->
</div>




<div style="
  font-family: Arial, sans-serif;
  background: #00e5ff;
  color: #0b1220;
  padding: 15px 20px;
  border-radius: 12px;
  line-height: 1.6;
  max-width: 650px;
">

**Question 11:**  
 Investigate what the padding option does. Discuss the different padding options and discuss which one you think is most suitable for the contrails. 

---

**Answer:**  
<!-- START OF ANSWER -->
<!-- END OF ANSWER -->
</div>

In [ ]:
p=0.5

transform_contrails_aug = A.Compose([
            A.Resize(128, 128),
            A.Rotate(limit=(-1.0*15, 15), border_mode=cv2.BORDER_REFLECT_101, p=0.5),
            A.Affine(translate_percent={'x': 0.1, 'y': 0.1}, border_mode=cv2.BORDER_REFLECT_101, p=p),
            A.GaussianBlur(blur_limit=3, sigma_limit=2, p=p),
            A.RandomBrightnessContrast(p=p),
            A.Perspective(scale=(0.05, 0.1), border_mode=cv2.BORDER_REFLECT_101, p=p),
            A.RGBShift(p=p),
            A.Normalize(mean=0.5, std=0.5, max_pixel_value=255.0),
        ])

transform_no_aug_contrails = A.Compose([
    A.Resize(128, 128),
    A.Normalize(mean=0.5, std=0.5, max_pixel_value=255.0),
])


EPOCHS = 100

#### Training the Models

The following code cells train the augmentations you came up with on the actual models - you are free to leave them to train over the break and then come back to them to see how the loss behaves with different validations. In the interest of time, however, we have pre-trained models for the later part of the session and provided the loss plots instead for the no-augmentation and augmentation model.

In [ ]:
# train_contrail_network(transform_no_aug_contrails, EPOCHS, "dice", "base", dataset="own", experiment_name="no_augmentation-dice")

![image.png](https://raw.githubusercontent.com/vdakov/contrail-segmentation/dev/images/contrail-loss-no-aug-200.png)

In [ ]:
# train_contrail_network(transform_contrails_aug, EPOCHS, "dice", "base", dataset="own", experiment_name="augmentation-dice")

![image.png](https://raw.githubusercontent.com/vdakov/contrail-segmentation/dev/images/contrail-loss-aug-200.png)

<div style="
  font-family: Arial, sans-serif;
  background: #00e5ff;
  color: #0b1220;
  padding: 15px 20px;
  border-radius: 12px;
  line-height: 1.6;
  max-width: 650px;
">

**Question 12:**  
Compare the loss plots of the no-augmentation vs. augmentation model. What is different? What about their predictions?

---

**Answer:**  
<!-- START OF ANSWER -->
<!-- END OF ANSWER -->
</div>

#### Are these predictions even good? 

Right now we have two trained contrail models - however, we have not defined ways to evaluate them. The reason is that getting a notion of how good the contrails model (or any deep learning model in industry for that matter) is hard. Here we show a naive approach of defining our metrics as "the number of matching pixels".

<div style="
  font-family: Arial, sans-serif;
  background: #00e5ff;
  color: #0b1220;
  padding: 15px 20px;
  border-radius: 12px;
  line-height: 1.6;
  max-width: 650px;
">

**Question 13:**  
Run the code cells below. What do you see about these predictions? Are these notions of accuracy reasonable?

---

**Answer:**  
<!-- START OF ANSWER -->
<!-- END OF ANSWER -->
</div>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

def evaluate_binary_image(input_img, label_img, threshold=0.5):
    # --- Load input image ---
    if isinstance(input_img, str):
        input_img = np.array(Image.open(input_img).convert('L')) / 255.0
    elif isinstance(input_img, np.ndarray) and input_img.ndim == 3:
        input_img = np.mean(input_img, axis=2)

    # --- Load label image (RGBA) ---
    if isinstance(label_img, str):
        label_img_pil = Image.open(label_img).convert('RGBA')
        label_arr = np.array(label_img_pil)
    elif isinstance(label_img, np.ndarray):
        label_arr = label_img

    # Alpha mask: 1 for visible pixels, 0 for transparent
    alpha_mask = label_arr[..., 3] > 0

    # Black pixels as label = 1
    black_mask = (label_arr[..., :3].sum(axis=2) == 0)
    label_bin = np.zeros(black_mask.shape, dtype=np.float32)
    label_bin[black_mask & alpha_mask] = 1.0  # only visible black pixels

    pred_bin = (input_img > threshold).astype(np.float32)

    accuracy = np.mean(pred_bin == label_bin)

    plt.figure(figsize=(12, 4))
    plt.subplot(1, 3, 1)
    plt.imshow(input_img, cmap='gray')
    plt.title("Input / Prediction")
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.imshow(label_bin, cmap='gray')
    plt.title("Ground Truth")
    plt.axis('off')

    plt.subplot(1, 3, 3)
    overlay = np.zeros((*pred_bin.shape, 3))
    overlay[..., 0] = label_bin  # red = GT
    overlay[..., 1] = pred_bin   # green = prediction
    overlay[..., 2] = label_bin * pred_bin  # blue = overlap
    plt.imshow(overlay, cmap='gray')
    plt.title(f"Overlay (GT=R, Pred=G)\nAccuracy={accuracy:.3f}")
    plt.axis('off')

    plt.show()

In [ ]:
evaluate_binary_image(np.zeros((235, 355)), "contrail-seg/data/goes/florida/mask/florida_2020_03_05_0101.png")

In [ ]:
evaluate_binary_image(np.ones((235, 355)), "contrail-seg/data/goes/florida/mask/florida_2020_03_05_0101.png")

In the following section, you will learn about how you can better evaluate your model.

---
# Metrics

![](https://raw.githubusercontent.com/vdakov/contrail-segmentation/dev/images/confusion_matrix.png)



"[Diagnostic testing diagram](https://en.wikipedia.org/wiki/Template:Diagnostic_testing_diagram)" by Wikipedia contributors / [CC BY-SA 4.0](https://creativecommons.org/licenses/by-sa/4.0/)

## Drawing Labels Yourself

In [ ]:
from widgets.precision_recall_paint import Painter 
IMAGE_PATH = "contrail-seg/data/goes/florida/image/florida_2020_03_05_0101.png"  
LABEL_PATH = "contrail-seg/data/goes/florida/mask/florida_2020_03_05_0101.png"

# Run the tool
p = Painter(IMAGE_PATH, LABEL_PATH)

### Comparing models

Now you have seen various ways to evaluate the models. Let's try to get our hands dirty with some actual models and their predictions:

| Metric        | Model A  | Model B|
| ------------- | ------------------------------------ | ------------------------------------ |
| **Accuracy**  | 0.94                                 | 0.89                                 |
| **Precision** | **0.92**                             | 0.48                                 |
| **Recall**    | 0.31                                 | **0.88**                             |
| **F1 Score**  | 0.46                                 | **0.62**                             |


<div style="
  font-family: Arial, sans-serif;
  background: #00e5ff;
  color: #0b1220;
  padding: 15px 20px;
  border-radius: 12px;
  line-height: 1.6;
  max-width: 1000px;
">

**Question 14:**  
You see the following model image, based on the input and label below. Judging off the metrics, which model does this belong to most likely - A or B? Fill it in for Model 1 and Model 2 below. 
<table>
  <tr>
    <td><b>Input</b><br><img src="https://raw.githubusercontent.com/vdakov/contrail-segmentation/dev/images/input-image-ex16.png" width="150"></td>
    <td><b>Label</b><br><img src="https://raw.githubusercontent.com/vdakov/contrail-segmentation/dev/images/ex-16-label-image.png" width="150"></td>
    <td><b>Model Prediction 1</b><br><img src="https://raw.githubusercontent.com/vdakov/contrail-segmentation/dev/images/ex-16-model-1.png" width="150"></td>
    <td><b>Model Prediction 2</b><br><img src="https://raw.githubusercontent.com/vdakov/contrail-segmentation/dev/images/ex-16-model-2.png" width="150"></td>
  </tr>
</table>


---

**Answer:**  
<!-- START OF ANSWER -->
<!-- END OF ANSWER -->
</div>

<div style="
  font-family: Arial, sans-serif;
  background: #00e5ff;
  color: #0b1220;
  padding: 15px 20px;
  border-radius: 12px;
  line-height: 1.6;
  max-width: 1000px;
">

**Question 15:**  
Which model would you prefer for the contrail task? A or B? Why?

---

**Answer:**  
<!-- START OF ANSWER -->
<!-- END OF ANSWER -->
</div>

### Evaluating the pre-trained models

In [ ]:
model_path_no_aug = "data/models/no_augmentation-dice-own-dice_base-200epoch.torch"
evaluate_contrail_model(model_path_no_aug)
_ = plot_pr_curve(model_path_no_aug)

In [ ]:
model_path_aug = "data/models/augmentation-dice-own-dice_base-200epoch.torch"
evaluate_contrail_model(model_path_aug)
_ = plot_pr_curve(model_path_aug)

<div style="
  font-family: Arial, sans-serif;
  background: #00e5ff;
  color: #0b1220;
  padding: 15px 20px;
  border-radius: 12px;
  line-height: 1.6;
  max-width: 1000px;
">

**Question 16:**  
The differences between the two real models are subtle. Describe what you see qualitatively between the two models. Which one would you prefer and why?

Can you think of a more suitable metric than precision and recall for comparing the two? Think once again about the task we have and why we do it: 
- We want to know how much contrails are in an image 
- We want to know where there contrails are

---

**Answer:**  
<!-- START OF ANSWER -->
<!-- END OF ANSWER -->
</div>

--- 

# Ethics: Responsibilities 

---

You have seen parts of the model pipeline already. You know how to think about critical aspects of your data before model training, during evaluation, and even more. 

**But what about how that model would be used**?   Unfortunately, in the real world, an application is rarely as simple as deploying the model - somebody is responsible for the effects caused by it - economic, social, etc. 

Assume that this assignment was provided by a stakeholder – say the Dutch government – because they want to use your tool to reduce this surface area. **The funder assumes that the minimization of contrail surface is good for the climate**. This means that it has effect on counteracting global warming. It is why they have given you this assigment. They want to deploy such a model to initialte policies that will reduce contrail surface area. This might result in the **prohibitation** of certain flight routes, for example. 



Unfortunately, research shows this may not be always the case, and there is some uncertainty about these effects. ([https://acp.copernicus.org/articles/24/9219/2024/](https://eur03.safelinks.protection.outlook.com/?url=https%3A%2F%2Facp.copernicus.org%2Farticles%2F24%2F9219%2F2024%2F&data=05%7C02%7CM.Sand%40tudelft.nl%7Cc76d0589bd4945603f1708de0d71de43%7C096e524d692940308cd38ab42de0887b%7C0%7C0%7C638962979860328000%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C0%7C%7C%7C&sdata=5CRoOnZviMYuDuqLT95zhF5r6S2O1n57RSHA8HuTbRQ%3D&reserved=0)): Since the physical processes that underlie these assumptions are very complex, there is a possibility that the contrary could be true. It could be the case that the bigger the surface area of the contrails, the more sun is being reflected before reaching and warming earth’s surface, which might in fact cause a “cooling effect” and is, therefore, beneficial for the climate. This uncertainty is difficult to quantify.

This poses an ethical challenge that we want you to discuss now: **What are your responsibilities as engineers in this regard?**

Do you have to report the uncertainties that you know about based on the research you have done in this project? Stronger even: Do you have a responsibility to discourage the intended use by the stakeholder (the government)?


In moral philosophy, we make an important distinction, which shall guide the first steps of your discussion. It is the distinction between backward- and forward-looking responsibility. Forward-looking responsibilities are responsibilities to ensure that certain negative events – harm, or damage – do not occur. Those are sometimes also named “duties” or “obligations”. But duties and obligations do not completely exhaust the concept of forward-looking responsibility: There are also certain character traits that are expected from people, which are forward-looking, for instance, to be faithful in relationships. For scientists, they need to be truthful, honest and reflexive – all of these are forward-looking virtues.

Backward-looking responsibilities – on the other hand – are invoked, when things have gone wrong, when damage has occurred or harm has been done. Then the harmed parties or third parties – the state, legal institutions – will hold one accountable. That means you have to answer for your actions, testify to whether you have known that those outcomes ensue or whether you might have even wanted those outcomes to come about directly or as a product of your actions.

In many case, in which someone has ignored a forward-looking responsibility and harm is caused, one is held accountable and made liable – one will be punished or blamed (held backward-looking responsible).

This roughly sketched conceptual distinctions shall inform some exercises that are best done in groups.


<div style="
  font-family: Arial, sans-serif;
  background: #00e5ff;
  color: #0b1220;
  padding: 15px 20px;
  border-radius: 12px;
  line-height: 1.6;
  max-width: 1000px;
">

**Question 17:**  
What is the difference between forward- and backward-looking responsibility? Provided an example for a machine learning case. If you have a partner, explain it to each other.

---

**Answer:**  
<!-- START OF ANSWER -->
<!-- END OF ANSWER -->
</div>




<div style="
  font-family: Arial, sans-serif;
  background: #00e5ff;
  color: #0b1220;
  padding: 15px 20px;
  border-radius: 12px;
  line-height: 1.6;
  max-width: 1000px;
">

**Question 18:**  
What kind of responsibility (active or passive) is at stake in the case just sketched?

---

**Answer:**  
<!-- START OF ANSWER -->
<!-- END OF ANSWER -->
</div>




Now the ball is in your court. What is your responsibility as an engineer? Please consult the following document: [Code of Ethics:The Royal Netherlands Society of Engineers](https://kivi.nl/_Resources/Persistent/5/3/3/8/5338c4ad1a3d5b95098c8313aec95307d29b4058/KIVI%20Code%20of%20Ethics%202018.pdf). It outlines basic responsibilities all engineers in the Netherlands should be familiar with. The principles apply to other countries as well. 

Your task is to read the document, think on the outlined responsibilities and relate them to the contrails case.


<div style="
  font-family: Arial, sans-serif;
  background: #00e5ff;
  color: #0b1220;
  padding: 15px 20px;
  border-radius: 12px;
  line-height: 1.6;
  max-width: 1000px;
">

**Question 19:**  
With your partner (if you have one), flip a coin and pick a thesis to defend from the ones below. If you are doing this alone, pick which thesis you agree with and write the arguments why while using the code of ethics. 

**Position 1**: Engineers are responsible for sharing possible uncertainties regarding related research that might impact how their products are being used by stakeholders. 
 
**Position 2**: Engineering teams are not responsible for sharing uncertainties that are related to their work, if it is not an integral part of the model itself. No matter the possible and predictably negative consequences of the utilization of their model. 

---

**Answer:**  
<!-- START OF ANSWER -->
<!-- END OF ANSWER -->
</div>




# Conclusion

Engineering projects are undertaken in the real world – they do not only describe and understand the world, they get initiated to change the world in important ways: For instance, by finding out, what better flight routes there are for airplanes and whether and how to impact current flight behavior. 

Because of that, engineering projects are always ethically relevant. They have social and environmental impacts; positive impacts if they are undertaken responsibly, or negative impacts if they are undertaken irresponsibly. 

Today, you have learned about the data selection process in machine learning, how we can make sure our models work well, and also that there are Codes of Conduct, established by professional engineering associations around the world. All of this should guide engineers to becoming better professionals. 

These documents – if you are uncertain about your responsibilities – can help you making better decisions. Oftentimes, however, their meaning must be continuously reinterpreted in light of new technologies and applications and interpretated in relation to actual cases: What does it mean to be transparent about uncertainty in the context of computational science? Who are the relevant stakeholders that need to be made aware of some consequences? How can we respect all cultural values equally in global engineering projects? 

To determine the meaning of these obligations and their scope, requires constant re-evaluation. For this, teamwork, a respectful and open dialogue about the scope and meaning of those terms is paramount. 

Such discussions will inadvertently also re-occur in your future professions. Today, you have practiced such dialogues in relation to a concrete case. We hope you have enjoyed this exercise; maybe the discussion changed your mind or let you see things with different eyes! That is the first step towards morally responsible engineering. 

#### Congratulations for rounding up this module! 